In [1]:
import numpy as np
import pandas as pd

In [2]:
df_clean2 = pd.read_csv('df_clean2.csv', low_memory=False)
df_clean2.keys()

Index(['Unnamed: 0', 'DateId', 'MemberId', 'OrderGroupCode',
       'TrackSourceTypeDef', 'TrackDeviceTypeDef', 'PayProfileTypeDef',
       'SalesOrderSlaveId', 'SalePageId', 'IsMajor', 'IsGift',
       'IsSalePageGift', 'Quantity', 'UnitPrice', 'PromotionDiscount',
       'ECouponId', 'ECouponDiscount', 'SalesOrderSlaveTotalPayment',
       'SalesOrderSlaveDateTime', 'SalesOrderReceiverId', 'City', 'District',
       'ZipCode', 'StoreName', 'StoreId', 'DeliverTypeDef', 'StatusDef',
       'ReturnGoodsDateId', 'CauseDef'],
      dtype='object')

 - 將地區分為北、中、東、南區

In [3]:
dict_region = {"北部":['臺北','台北','新北','基隆','宜蘭','桃園','新竹'],
               "中部":['苗栗','臺中','台中','彰化','南投','雲林'],
               "南部":['嘉義','臺南','台南','高雄','屏東','澎湖'],
               "東部":['花蓮','台東','臺東']
              }

In [10]:
m_state = df_clean2[df_clean2['City'] == df_clean2['City']]  #過濾nan
m_state.head()

,Unnamed: 0,DateId,MemberId,OrderGroupCode,TrackSourceTypeDef,TrackDeviceTypeDef,PayProfileTypeDef,SalesOrderSlaveId,SalePageId,IsMajor,...,SalesOrderReceiverId,City,District,ZipCode,StoreName,StoreId,DeliverTypeDef,StatusDef,ReturnGoodsDateId,CauseDef
0,0,20150703,1388133,b'\xd0_\x8eby\xe8\x88\xa5\x85\xe4\x15\nMC\xeaO...,Web,Mobile,SevenEleven,4457777,1242871,True,...,1480890,嘉義縣,水上鄉,608.0,中庄門市,910828,SevenEleven,Finish,NaN,NaN
1,1,20150703,1388133,b'\xd0_\x8eby\xe8\x88\xa5\x85\xe4\x15\nMC\xeaO...,Web,Mobile,SevenEleven,4457778,1242880,True,...,1480890,嘉義縣,水上鄉,608.0,中庄門市,910828,SevenEleven,Finish,NaN,NaN
2,2,20150703,1388133,b'\xd0_\x8eby\xe8\x88\xa5\x85\xe4\x15\nMC\xeaO...,Web,Mobile,SevenEleven,4457779,1242880,True,...,1480890,嘉義縣,水上鄉,608.0,中庄門市,910828,SevenEleven,Finish,NaN,NaN
3,3,20150703,1388133,b'\xd0_\x8eby\xe8\x88\xa5\x85\xe4\x15\nMC\xeaO...,Web,Mobile,SevenEleven,4457780,1175574,True,...,1480890,嘉義縣,水上鄉,608.0,中庄門市,910828,SevenEleven,Finish,NaN,NaN
4,4,20150703,365787,"b""\x04?}pe$m\x7f\x85\xa0y\x81'e5\xca\xbb\xf9\x...",Web,Mobile,SevenEleven,4457791,1194488,True,...,1480896,桃園市,中壢區,320.0,復華門市,114071,SevenEleven,Finish,NaN,NaN


In [5]:
def get_keys(d, seri_state):
    
    result = list()
    for state in seri_state:
        locate_flag = 0
        for key,value in d.items():
            for a in value:            
                if a in state:
                    result.append(key)
                    locate_flag = 1
                    break
            if locate_flag == 1:
                break
        if locate_flag == 0:
            result.append('not_define')
    return result
    
get_keys(dict_region, ['桃園'])

['北部']

In [11]:
m_state['Area'] = get_keys(dict_region, m_state['City'])

/home/dongdong/.local/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [12]:
m_state =  m_state[m_state['Area'] != 'not_define'] # 

In [14]:
# 取出 "地區、城市、購買數量、物品編號、時間"
m_state['new_DateId']=pd.to_datetime(m_state['DateId'].astype(str), format='%Y%m%d')
a0 = pd.Series(m_state['new_DateId'])
a1 = pd.Series(m_state['SalePageId'])
a2 = pd.Series(m_state['Quantity'])
a3 = pd.Series(m_state['City'])
a4 = pd.Series(m_state['Area'])

data = {'new_DateId':a0,'SalePageId':a1,'Quantity':a2,'City':a3,'Area':a4}
item_loc = pd.DataFrame(data)
item_loc.head()

,Area,City,Quantity,SalePageId,new_DateId
0,南部,嘉義縣,1,1242871,2015-07-03
1,南部,嘉義縣,1,1242880,2015-07-03
2,南部,嘉義縣,1,1242880,2015-07-03
3,南部,嘉義縣,1,1175574,2015-07-03
4,北部,桃園市,1,1194488,2015-07-03


In [15]:
city_num = item_loc.groupby(by='City')['Quantity'].sum()
city_num

City
南投縣      74939
台中市     815918
台北市     705464
台南市     407470
台東縣      35217
嘉義市      55276
嘉義縣      68262
基隆市      89367
宜蘭縣      87111
屏東縣     125897
彰化縣     201898
新北市    1196609
新竹市     148414
新竹縣     129924
桃園市     599553
澎湖縣      20190
花蓮縣      69475
苗栗縣     108647
雲林縣      88407
高雄市     580556
Name: Quantity, dtype: int64

In [28]:
name = []
s0 = []
s1 = []
for i in range(len(city_num)):
    item_sum = ((item_loc[item_loc['City'].str.contains(city_num.keys()[i])==True]).groupby(by='SalePageId')['Quantity'].sum()).sort_values(ascending = False).head()
    for j in range(0,5):
        name.append((city_num.keys())[i])
    s0.append(pd.Series(item_sum.keys()))
    s1.append(pd.Series(np.array(item_sum)))
b2 = pd.Series(name)
b1 = pd.Series(s1)
b0 = pd.Series(s0)

In [29]:
t0 = []
t1 = []
for i in range((len(b0))):
    t0 = np.hstack((t0,np.array(b0[i])))
    t1 = np.hstack((t1,np.array(b1[i])))
s0 = pd.Series(t0)
s1 = pd.Series(t1)

In [30]:
# 找出每個縣市最喜歡購買前五名的物品
dataa = {'City':b2,'ItemId':s0,'Quantity':s1}
item_num_loc = pd.DataFrame(dataa)

In [31]:
item_num_loc.iloc[0:10]

,City,ItemId,Quantity
0,南投縣,1830694.0,393.0
1,南投縣,2336461.0,384.0
2,南投縣,2294453.0,317.0
3,南投縣,3830498.0,275.0
4,南投縣,1119387.0,271.0
5,台中市,2336461.0,4218.0
6,台中市,2294453.0,4018.0
7,台中市,1830694.0,3737.0
8,台中市,1119387.0,3399.0
9,台中市,3830498.0,3063.0


 - #### 因為有些城市購買的物品相似，所以還是很難看出城市之間不同喜好 ； 因此下一步想做每個城市每個月物品購買隨時間的變化

In [44]:
name = []
item_year = []
item_month =[]
s0 = []
s1 = []
year = ['5','6','7','8']
month = ['01','02','03','04','05','06','07','08','09','10','11','12']
for i in range(len(city_num)):
    rr = (item_loc['City'].str.contains(city_num.keys()[i])==True]).sort_values(['new_DateId'],ascending = True)
    for q in range(len(year)):
        y = year[q]
        for r in range(len(month)):
            m =  month[r]
            item_sum = (rr[(rr['new_DateId']>'201'+y+'-'+m+'-01') & (rr['new_DateId']<('201'+y+'-'+m+'-31'))].groupby(by='SalePageId')['Quantity'].sum()).sort_values(ascending = False).head()
            for j in range(0,5):
                name.append((city_num.keys())[i])
                item_year.append(('201'+y))
                item_month.append(m)
            s0.append(pd.Series(item_sum.keys()))
            s1.append(pd.Series(np.array(item_sum)))
b4 = pd.Series(item_year)
b3 = pd.Series(item_month)
b2 = pd.Series(name)
b1 = pd.Series(s1)
b0 = pd.Series(s0)

SyntaxError: invalid syntax (<ipython-input-44-f8d5d953a735>, line 9)

In [ ]:
len(b0),len(b1),len(b2),len(b3),len(b4)

In [ ]:
empty_list = []
value_list = []
value_list1 = []
b2_after = []
b3_after = []
b4_after = []
for i in range(len(b0)):
    gg = np.array(b0[i])
    if len(gg) < 4:        
        empty_list.append(i)
    if len(gg) > 4:
        value_list.append(b0[i])
        value_list1.append(b1[i])
        for j in range(0,5):
            b2_after.append(b2[(i*5+j)])
            b3_after.append(b3[(i*5+j)])
            b4_after.append(b4[(i*5+j)])

In [ ]:
t0 = []
t1 = []
for i in range((len(value_list))):
    t0 = np.hstack((t0,np.array(value_list[i])))
    t1 = np.hstack((t1,np.array(value_list1[i])))
s0 = pd.Series(t0)
s1 = pd.Series(t1)
s2 = pd.Series(b2_after)
s3 = pd.Series(b3_after)
s4 = pd.Series(b4_after)

In [ ]:
dataaa = {'ItemId':s0,'Quantity':s1,'City':s2,'month':s3,'year':s4}
item_num_loc_ = pd.DataFrame(dataaa)

In [ ]:
item_num_loc_.head()

In [ ]:
i = 0
kk = item_num_loc_[item_num_loc_['City'].str.contains((city_num.keys())[i])==True]
kk

In [ ]:
ItemId_top = []
for i in range(int(len(kk)/5)):
    ItemId_top.append((kk.iloc[[(i*5)]])['ItemId'][(i*5)])

In [ ]:
ItemId_top[0:100]

 - #### 想看物品編號對應的商品，比較容易看出關係

In [ ]:
raw_data = pd.read_csv('Ntu_Orders.csv')
df = pd.read_csv('Orders.csv',header = 0,sep = ",")

In [ ]:
a =[1162433.0,1194488.0,1270622.0,1348360.0,1460234.0,1504091.0,1597130.0,1597130.0,1783258.0,
    1910591.0,1862588.0,1830694.0,1119387.0,1119387.0,2294453.0,2336461.0,2294453.0]

In [ ]:
i= 16
aaa = raw_data.loc[raw_data['SalePageId'] == a[i]]
aaa

 - ### 可能物品編號方式不同，因此找不到相對應的物品名稱

 - ## 想觀察購買物品的數量及種類是否會隨天氣而有所變化
 - ### Step1 : 做每個城市每天購買的物品最多的
 - ### Step2 : 與天氣資訊做結合

In [ ]:
name = []
item_year = []
item_month =[]
item_day =[]
s0 = []
s1 = []
year = ['5','6','7','8']
month = ['01','02','03','04','05','06','07','08','09','10','11','12']
day = ['01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31']

In [ ]:
for i in range(len(city_num)): # 得到城市照日期排的資訊
    rr = (item_loc[item_loc['City'].str.contains((city_num.keys())[i])==True]).sort_values(['new_DateId'],ascending = True)
    for q in range(len(year)):
        y = year[q]
        for r in range(len(month)):
            m =  month[r]
            for s in range((len(day)-2)): # 得到某日購買項目最多的前兩名
                d = day[s]
                d1 = day[s+1]
                itemm = (rr[(rr['new_DateId']>('201'+y+'-'+m+'-'+d)) & (rr['new_DateId']<('201'+y+'-'+m+'-'+d1))].groupby(by='SalePageId')['Quantity'].sum()).sort_values(ascending = False)
                item_sum = (itemm.head(2))
                item_mean = itemm.sum()
                if ((len(item_sum) > 1)): # 判斷第一和第二數量，第一大於第二將資料收集起來
                    if (item_sum.iloc[0] > item_sum.iloc[1]):
                        item_sum = item_sum.iloc[[0]]
                        name.append((city_num.keys())[i])
                        item_year.append(('201'+y))
                        item_month.append((m+d))
                        s0.append(pd.Series(item_sum.keys()))
                        ave = (np.array(item_sum)/np.array(item_mean))
                        s1.append(ave[0])
b5 = pd.Series(item_year)
b4 = pd.Series(item_month)
b3 = pd.Series(item_day)
b2 = pd.Series(name)
b1 = pd.Series(s1)
b0 = pd.Series(s0)

In [ ]:
empty_list = []
value_list = []
value_list1 = []
b2_after = []
b3_after = []
b4_after = []
for i in range(len(b0)):
    gg = np.array(b0[i])
    if len(gg) <= 0:        
        empty_list.append(i)
    if len(gg) > 0:
        value_list.append(b0[i])
        value_list1.append(b1[i])
        b2_after.append(b2[(i)])
        b3_after.append(b4[(i)])
        b4_after.append(b5[(i)])

In [ ]:
t0 = []
t1 = []
for i in range((len(value_list))):
    t0 = np.hstack((t0,np.array(value_list[i])))
    t1 = np.hstack((t1,np.array(value_list1[i])))
s0 = pd.Series(t0)
s1 = pd.Series(t1)
s2 = pd.Series(b2_after)
s3 = pd.Series(b3_after)
s4 = pd.Series(b4_after)

In [ ]:
dataaaa = {'ItemId':s0,'Quantity':s1,'City':s2,'month':s3,'year':s4}
item_num_loc_day = pd.DataFrame(dataaaa)

In [ ]:
item_num_loc_day

In [ ]:
item_num_loc_day.to_csv('item_num_loc_day.csv', sep=',', encoding='utf-8-sig')
# 每個城市每日購買數量最多的品項

In [ ]:
item_num_loc_day = pd.read_csv('item_num_loc_day.csv')

In [ ]:
dict_region = {"北部":['臺北','台北','新北','基隆','宜蘭','桃園','新竹'],
               "中部":['苗栗','臺中','台中','彰化','南投','雲林'],
               "南部":['嘉義','臺南','台南','高雄','屏東','澎湖'],
               "東部":['花蓮','台東','臺東']
              }

In [ ]:
def get_keys(d, seri_state):
    
    result = list()
    for state in seri_state:
        locate_flag = 0
        for key,value in d.items():
            for a in value:            
                if a in state:
                    result.append(key)
                    locate_flag = 1
                    break
            if locate_flag == 1:
                break
        if locate_flag == 0:
            result.append('not_define')
    return result
    
get_keys(dict_region, ['桃園'])

In [ ]:
item_num_loc_day['Area'] = get_keys(dict_region, item_num_loc_day['City'])
item_num_loc_day =  item_num_loc_day[item_num_loc_day['Area'] != 'not_define'] # 

# a4 = pd.Series(item_num_loc_day_temp['Area'])

# data = {'new_DateId':a0,'SalePageId':a1,'Quantity':a2,'City':a3,'Area':a4}

In [ ]:
item_num_loc_day.head(-100)

In [ ]:
item_num_loc_day.to_csv('item_num_loc_day.csv', sep=',', encoding='utf-8-sig')

 - ### 合併天氣資料

In [ ]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from urllib.parse import quote
import time

In [ ]:
# 用pd.read_html抓取資料表格
station_info = pd.read_csv('station_info.csv')

In [ ]:
station_info.drop('Unnamed: 0',axis = 1,inplace=True)   #axis:去行（0）或列（1）

In [ ]:
stationIndexList = station_info.columns

## [ 降雨量 ,  溫度 ] = getClimate(site,date,mode)
### 需先執行
- station_info = pd.read_csv('station_info.csv')
- station_info.drop('Unnamed: 0',axis = 1,inplace=True)
- stationIndexList = station_info.columns

### mode=1 : 較慢但較準，採取人工測站與無人測站，且加入delay防鎖IP
### mode=2 : 較快但不準，只採取人工測站，未加入delay

In [ ]:
def getClimate(site,date,mode):
    print('Site : ' + site)
    print('Date : ' + date)
    
    precpAvgList = list()
    tempAvgList = list()
    
    #台-->臺
    if site[0] == '台':
        site = '臺'+site[1:] #字串無法直接指定, ex:site[k] = '臺'
            
    if mode == 1: 
        for i in range(len(stationIndexList)):       
            
            #此地區每個測站
            if station_info[stationIndexList[i]][2] == site:
                
                #需要delay才不會被擋
                time.sleep(0.11) 
                
                #去指定網址爬取                
                print(station_info[stationIndexList[i]][4] + date) #網址
                temp=pd.read_html(station_info[stationIndexList[i]][4] + date)
                
                 #此測站24小時的資料
                for j in range(len(temp[1][10][2:])):
                    
                    #清空
                    precpList = list()
                    tempList = list() 
                    
                    #讀取，若是X則跳過
                    try:
                        precpList.append(np.float(temp[1][10][j+2]))
                        tempList.append(np.float(temp[1][3][j+2]))
                    except:
                        continue
                        
                    #求得此測站平均
                    precpAvg = np.average(precpList)
                    tempAvg = np.average(tempList)
                    
                #此地區所有測站值
                if precpAvg == precpAvg:  #確認是否為nan
                    precpAvgList.append(precpAvg)
                if tempAvg == tempAvg:
                    tempAvgList.append(tempAvg)
                    
        #算出所有測站平均回傳 
        print("By mode 1-->")
        print("測站的降雨量日平均值 : " + precpAvgList)
        print("測站的溫度日平均值 : " + tempAvgList)
        
        return np.average(precpAvgList), np.average(tempAvgList)
    
    else:
        for i in range(len(stationIndexList)):
            
            #此地區每個人工測站
            if (station_info[stationIndexList[i]][2] == site) and stationIndexList[i].isdigit():
                
                #需要delay才不會被擋
                time.sleep(0.11) 
                
                #去指定網址爬取
                print(station_info[stationIndexList[i]][4] + date) #網址
                temp=pd.read_html(station_info[stationIndexList[i]][4] + date)
    
                 #此測站24小時的資料
                for j in range(len(temp[1][10][2:])):
                    #清空
                    precpList = list()
                    tempList = list() 
                    
                    #讀取，若是X則跳過
                    try:
                        precpList.append(np.float(temp[1][10][j+2]))
                        tempList.append(np.float(temp[1][3][j+2]))
                    except:
                        continue
                        
                    #求得此測站平均
                    precpAvg = np.average(precpList)
                    tempAvg = np.average(tempList)
                
                #此地區所有測站值
                if precpAvg == precpAvg:  #確認是否為nan
                    precpAvgList.append(precpAvg)
                if tempAvg == tempAvg:
                    tempAvgList.append(tempAvg)
#             else:
#                 break
        #算出人工測站平均回傳
        print("By mode 2-->")
        print("測站的降雨量日平均值 : " + str(precpAvgList))
        print("測站的溫度日平均值 : " + str(tempAvgList))
        
        #如果無人工測站，改用mode=1
        if not((np.average(precpAvgList) == np.average(precpAvgList)) or (np.average(tempAvgList) == np.average(tempAvgList))):
            print("mode 2 is fail")

            precpAvgList = list()
            tempAvgList = list()
            for i in range(len(stationIndexList)):       
            
                #此地區每個測站
                if station_info[stationIndexList[i]][2] == site:

                    #需要delay才不會被擋
                    time.sleep(0.11) 

                    #去指定網址爬取
                    print(station_info[stationIndexList[i]][4] + date) #網址
                    temp=pd.read_html(station_info[stationIndexList[i]][4] + date)

                     #此測站24小時的資料
                    for j in range(len(temp[1][10][2:])):

                        #清空
                        precpList = list()
                        tempList = list()

                        #讀取，若是X則跳過
                        try:
                            precpList.append(np.float(temp[1][10][j+2]))
                            tempList.append(np.float(temp[1][3][j+2]))
                        except:
                            continue

                    #求得此測站平均
                    precpAvg = np.average(precpList)
                    tempAvg = np.average(tempList)

                    #此地區所有測站值
                    if precpAvg == precpAvg:  #確認是否為nan
                        precpAvgList.append(precpAvg)
                    if tempAvg == tempAvg:
                        tempAvgList.append(tempAvg)

            #算出所有測站平均回傳
            print("mode 2 改 mode 1-->")            
            print("測站的降雨量日平均值 : " + str(precpAvgList))
            print("測站的溫度日平均值 : " + str(tempAvgList))
            print("")
        return np.average(precpAvgList), np.average(tempAvgList)
        

In [ ]:
len(item_num_loc_day_temp[8885:]),len(item_num_loc_day)

In [ ]:
item_num_loc_day = item_num_loc_day_temp[13137:]

In [ ]:
item_num_loc_day.head()

In [ ]:
loc_rain_2 = []
loc_temp_2 = []
for i in range(len(item_num_loc_day)):    
    hy = str(item_num_loc_day['year'][i+13137])
    hm = str(item_num_loc_day['month'][i+13137])
    if len(hm) == 4:
        a_date = hy+'-'+hm[0:2]+'-'+hm[-2:]
    if len(hm) == 3:
        a_date = hy+'-0'+hm[0]+'-'+hm[-2:]
    a_loc = item_num_loc_day['City'][i+13137]        
    dd = getClimate(a_loc,a_date,2)
    loc_rain_2.append(dd[0])    
    loc_temp_2.append(dd[1])

In [ ]:
tt1_2 = pd.Series(loc_rain_2)
tt2_2 = pd.Series(loc_temp_2)
data_2 = {'rain_2':tt1_2,'temp_2':tt2_2}
item_num_loc_day_temp_2 = pd.DataFrame(data_2)
item_num_loc_day_temp_2.to_csv('item_num_rain_temp_2.csv',sep = ',',encoding='utf-8-sig')
item_num_loc_day_temp_2.head(100)

In [ ]:
len(item_num_loc_day_temp_2)

In [ ]:
tt1 = pd.Series(loc_rain)
tt2 = pd.Series(loc_temp)
item_num_loc_day['rain'] = tt1
item_num_loc_day['temp'] = tt2

In [ ]:
item_num_loc_day.head()

In [ ]:
s0 = pd.Series(t0)
s1 = pd.Series(t1)
s2 = pd.Series(b2_after)
s3 = pd.Series(b3_after)
s4 = pd.Series(b4_after)
s5 = pd.Series(loc_rain)
s6 = pd.Series(loc_temp)

In [ ]:
len(s0),len(s1),len(s2),len(s3),len(s4),len(s5),len(s6)

In [ ]:
dataaaaa = {'ItemId':s0,'Quantity':s1,'City':s2,'month':s3,'year':s4,'rain':s5,'temp':s6}
item_num_loc_day_temp = pd.DataFrame(dataaaaa)

In [ ]:
item_num_loc_day_temp

In [ ]:
item_num_loc_day.to_csv('item_num_rate_loc_day_temp.csv',sep = ',',encoding='utf-8-sig')

In [ ]:
for i in range(len(item_num_loc_day_temp_2)):
    item_num_loc_day_temp['rain'][i+13137] = tt1_2[i]
    item_num_loc_day_temp['temp'][i+13137] = tt2_2[i]

In [ ]:
item_num_loc_day = pd.read_csv('item_num_rate_loc_day_temp.csv')

In [ ]:
item_num_loc_day_temp = item_num_loc_day

In [ ]:
# Item_ID num
pp = item_num_loc_day_temp.groupby(by='ItemId')['Quantity'].sum()
len(pp)

In [ ]:
# Check data if have nan
na_rain = (item_num_loc_day_temp[item_num_loc_day_temp.isnull().any(axis=1)]['rain'])
na_temp = (item_num_loc_day_temp[item_num_loc_day_temp.isnull().any(axis=1)]['temp'])
len(na_rain),len(na_temp)

In [ ]:
na_rain.head(100)

In [ ]:
a = [9085,9177,9364,9567,12470,12565,12926]
for i in range(len(a)):
    n = a[i]
    item_num_loc_day_temp['rain'][n] = (item_num_loc_day_temp.iloc[[n-1]]['rain'][n-1]+item_num_loc_day_temp.iloc[[n+1]]['rain'][n+1])/2
    item_num_loc_day_temp['temp'][n] = (item_num_loc_day_temp.iloc[[n-1]]['temp'][n-1]+item_num_loc_day_temp.iloc[[n+1]]['temp'][n+1])/2

In [ ]:
item_num_loc_day_temp[item_num_loc_day_temp.isnull().any(axis=1)]['rain']

 - ## Use RNN_LSTM to train

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.examples.tutorials.mnist import input_data
tf.set_random_seed(1)   # set random seed

In [ ]:
item_num_loc_day_temp = pd.read_csv('item_num_rate_loc_day_temp.csv')

 - train data

In [ ]:
nj = []
s5 = pd.Series(np.array(item_num_loc_day_temp['rain']))
s6 = pd.Series(np.array(item_num_loc_day_temp['temp']))
datb = {'rain':s5,'temp':s6}
datbb = pd.DataFrame(datb)
nj = np.array(datbb.head(661))

In [ ]:
# 以南投縣為例[0:661]
# t = item_num_loc_day_temp.head(661)['year']+item_num_loc_day_temp.head(661)['month'] # 順序
t = item_num_loc_day_temp.head(661)['year']+item_num_loc_day_temp.head(661)['month']
y_real  =  nj   # 天氣(雨及溫度)
plt.plot(t, y_real, 'b')
plt.show()

In [ ]:
# hyperparameters
lr = 0.001                  # learning rate
training_iters = 100        # train step 上限
batch_size = 7            
n_inputs = 3                # data_feature
n_steps = 7                 # time steps
n_hidden_units = 128        # neurons in hidden layer
n_classes = 1774            # Item_ID num 

In [ ]:
# create x y placeholder
x = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.float32, [None, n_classes])

 - Functions of weight and bias

In [ ]:
weights = {
    # shape (3, 128)
    'in': tf.Variable(tf.random_normal([n_inputs, n_hidden_units])),
    # shape (128, 1774)
    'out': tf.Variable(tf.random_normal([n_hidden_units, n_classes]))
}
biases = {
    # shape (128, )
    'in': tf.Variable(tf.constant(0.1, shape=[n_hidden_units, ])),
    # shape (1774, )
    'out': tf.Variable(tf.constant(0.1, shape=[n_classes, ]))
}

 - Encoder inputs and outputs

In [ ]:
def RNN(X, weights, biases):
    # 原始的 X 是 3 维数据, 我们需要把它变成 2 维数据才能使用 weights 的矩阵乘法
    # X ==> (128 batches * 7 steps, 3 inputs)
    X = tf.reshape(X, [-1, n_inputs])

    # X_in = W*X + b
    X_in = tf.matmul(X, weights['in']) + biases['in']
    # X_in ==> (128 batches, 7 steps, 128 hidden) 换回3维
    X_in = tf.reshape(X_in, [-1, n_steps, n_hidden_units])

In [ ]:
# 使用 basic LSTM Cell.
lstm_cell = tf.contrib.rnn.BasicLSTMCell(n_hidden_units, forget_bias=1.0, state_is_tuple=True)
init_state = lstm_cell.zero_state(batch_size, dtype=tf.float32) # 初始化全零 state

In [ ]:
outputs, final_state = tf.nn.dynamic_rnn(lstm_cell, X_in, initial_state=init_state, time_major=False)

In [ ]:
# outputs generate
w_out = _weight([n_hiddens, n_output])
b_out = _bias([n_output])
preds = [tf.add(tf.matmul(decoder_outputs[i], w_out), b_out, name='pred') for i in range(dec_len)]
preds, y

 - Loss and optimizer

In [ ]:
output_loss = 0
for _y, _Y in zip(preds, y):
    output_loss += tf.reduce_mean(tf.nn.l2_loss(_y - _Y))

l_r = 1e-3
update_step = tf.train.AdamOptimizer(l_r).minimize(output_loss)

 - Evaluation function (plotting)

In [ ]:
def evaluate(sess):
    batch_x = [y_real[i].reshape([-1, 1]) for i in range(5)]
    batch_y = [y_real[i+5].reshape([-1, 1]) for i in range(5)]
    feed_dict = {x[j]:batch_x[j] for j in range(enc_len)}
    feed_dict.update({y[j]:batch_y[j] for j in range(enc_len)})
    y_pred1 = np.array(sess.run(preds, feed_dict=feed_dict))

    batch_x = y_pred1
    batch_y = [y_real[i+10].reshape([-1, 1]) for i in range(5)]
    feed_dict = {x[j]:batch_x[j] for j in range(enc_len)}
    feed_dict.update({y[j]:batch_y[j] for j in range(enc_len)})
    y_pred2 = np.array(sess.run(preds, feed_dict=feed_dict))

    y_pred = np.vstack([y_pred1, y_pred2]).reshape([-1])
    print(len([y_real[4]]+list(y_pred)),len(list(y_pred)))
    plt.plot(np.arange(21), y_real[0:21], 'b')
    plt.plot(np.arange(21), [y_real[4]]+list(y_pred), '--r')    
    plt.legend(['Real', 'Prediction'])
    
def evaluate2(sess):
    batch_x = [y_real[i+3].reshape([-1, 1]) for i in range(5)]
    batch_y = [y_real[i+5+3].reshape([-1, 1]) for i in range(5)]
    feed_dict = {x[j]:batch_x[j] for j in range(enc_len)}
    feed_dict.update({y[j]:batch_y[j] for j in range(enc_len)})
    y_pred1 = np.array(sess.run(preds, feed_dict=feed_dict))

    batch_x = y_pred1
    batch_y = [y_real[i+10+3].reshape([-1, 1]) for i in range(5)]
    feed_dict = {x[j]:batch_x[j] for j in range(enc_len)}
    feed_dict.update({y[j]:batch_y[j] for j in range(enc_len)})
    y_pred2 = np.array(sess.run(preds, feed_dict=feed_dict))

    y_pred = np.vstack([y_pred1, y_pred2]).reshape([-1])
    plt.plot(np.arange(15), y_real[0+3:15+3], 'b')
    plt.plot(np.arange(11)+4, [y_real[4+3]]+list(y_pred), '--r')
    plt.legend(['Real', 'Prediction']) 
    
def evaluate3(sess):
    batch_x = [y_real[i+5].reshape([-1, 1]) for i in range(5)]
    batch_y = [y_real[i+5+5].reshape([-1, 1]) for i in range(5)]
    feed_dict = {x[j]:batch_x[j] for j in range(enc_len)}
    feed_dict.update({y[j]:batch_y[j] for j in range(enc_len)})
    y_pred1 = np.array(sess.run(preds, feed_dict=feed_dict))

    batch_x = y_pred1
    batch_y = [y_real[i+10+5].reshape([-1, 1]) for i in range(5)]
    feed_dict = {x[j]:batch_x[j] for j in range(enc_len)}
    feed_dict.update({y[j]:batch_y[j] for j in range(enc_len)})
    y_pred2 = np.array(sess.run(preds, feed_dict=feed_dict))

    y_pred = np.vstack([y_pred1, y_pred2]).reshape([-1])
    plt.plot(np.arange(15), y_real[0+5:15+5], 'b')
    plt.plot(np.arange(11)+4, [y_real[4+5]]+list(y_pred), '--r')
    plt.legend(['Real', 'Prediction'])

 - Training

In [ ]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
epoch = 100
batch_size = n
loss_list = list()

In [ ]:
# Initial prediction
plt.figure(figsize=[17, 4])
plt.subplot(131)
evaluate(sess)
plt.subplot(132)
evaluate2(sess)
plt.subplot(133)
evaluate3(sess)
plt.show()

In [ ]:
plt.figure(figsize=[17, 4])

evaluate(sess)